In [26]:
%%capture
%pip install yfinance
%pip install pandas
%pip install matplotlib
%pip install plotly
%pip install numpy
%pip install statsmodels

In [27]:
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

***CURRENTLY OWNED STOCKS***

In [24]:
amazon = yf.Ticker("AMZN")
rivian = yf.Ticker("RIVN")
nvidia = yf.Ticker("NVDA")
oatly = yf.Ticker("OTLY")
meta = yf.Ticker("META")

In [28]:
amazon_history = amazon.history(period = "max")
rivian_history = rivian.history(period = "max")
nvidia_history = nvidia.history(period = "max")
oatly_history = oatly.history(period = "max")
meta_history = meta.history(period = "max")